# Data preprocessing and identification

In [2]:
from imblearn.over_sampling import BorderlineSMOTE
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
import math
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from lightgbm import LGBMClassifier
text_neg = []
text_pos = []


def loaddata(path):
    numwords = []
    file = open(path, encoding="utf-8")
    texts_projects = []
    label_projects = []
    for line in file.readlines():
        line = line.strip("\n")
        a = line[13:].lower()
        if a == '':
            continue
        if a == '   ':
            continue
        if a == ' ':
            continue
        if a == '  ':
            continue
        texts_projects.append(a)

        counter = len(a.split(" "))
        numwords.append(counter)
        if line[4:12] == "negative":
            label_projects.append(0)
            text_neg.append(a)
        else:
            label_projects.append(1)
            text_pos.append(a)
    return texts_projects, label_projects, numwords


def dataSplit(target):
    path = './data'
    files = os.listdir(path)
    texts = []
    labels = []
    numwords = []
    for file in files:
        if file != target:
            print(file)
            text, label, numword = loaddata(r'./data/' + file)
            texts += text
            labels += label
            numwords += numword
    text1, label1, numwords1 = loaddata(r'./data/' + target)
    return texts, labels, numwords, text1, label1, numwords1


target = 'Arg_processing.csv'
vocabulary = {"this": 0, "the": 1, "not": 2}
x_tr, y_train, numwords, x_te, y_test, numwords1 = dataSplit(target)
idf = TfidfVectorizer(lowercase=False, max_features=1000, stop_words='english')
x_train = idf.fit_transform(x_tr)
x_test = idf.transform(x_te)
smo = BorderlineSMOTE()
x_train, y_train = smo.fit_resample(x_train, y_train)
smo.fit(x_train, y_train)
from lightgbm import LGBMClassifier
# gbm0=LGBMClassifier(max_depth= 4, num_leaves= 50,learning_rate=0.1, n_estimators=800)
gbm0=LGBMClassifier(max_depth= 3, num_leaves= 50,learning_rate=0.1, n_estimators=800)
eval_set = [(x_test, y_test)]
gbm0.fit(x_train, y_train, eval_set=[(x_test, y_test)])
# gbm0=GradientBoostingClassifier(n_estimators=80, max_depth=6, min_samples_split =1400)
y_predict = gbm0.predict(x_test)
y_pred = gbm0.predict(x_test)
y_predprob = gbm0.predict_proba(x_test)[:, 1]
print("Accuracy:%.4f" % metrics.accuracy_score(y_test, y_pred))
tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_predict).ravel()
rec = tp / (tp + fn)
Recall = tp / (tp + fn)
specificity = tn / (fp + tn)
precision = tp / (tp + fp)
gmeans = math.sqrt(specificity * rec)
f1 = 2 * precision * rec / (precision + rec)

print("recall:", rec)

print("precision:", precision)

print("f1:", f1)
print(y_pred)


Ant_processing.csv
Col_processing.csv
EMF_processing.csv
Hib_processing.csv
JEd_processing.csv
JFr_processing.csv
JMe_processing.csv
JRu_processing.csv
SQu_processing.csv


D:\app\anaconda3\envs\py37\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
D:\app\anaconda3\envs\py37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Accuracy:0.9590
recall: 0.8853503184713376
precision: 0.8481355932203389
f1: 0.8663434903047091
[1 1 1 ... 0 0 0]


# LIME

In [3]:
idx = 2
gbm0.predict_proba(x_test[idx])

array([[0.10757569, 0.89242431]])

In [4]:
from lime import lime_text
from sklearn.pipeline import make_pipeline

c = make_pipeline(idf, gbm0)
print(c.predict_proba([x_te[0]]))

from lime.lime_text import LimeTextExplainer

class_names = ['negative', 'positive']
explainer = LimeTextExplainer(class_names=class_names)
idx = 8000
exp = explainer.explain_instance(x_te[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print(c.predict_proba([x_te[idx]]))
print('Probability =', c.predict_proba([x_te[idx]])[0, 1])
print('True class: %s' % class_names[y_test[idx]])
exp.as_list()
print(exp.as_list())

print('Original prediction:', gbm0.predict_proba(x_test[idx])[0, 1])
tmp = x_test[idx].copy()
tmp[0, idf.vocabulary_['todo']] = 0
print('Prediction removing some features:', gbm0.predict_proba(tmp)[0, 1])
print('Difference:', gbm0.predict_proba(tmp)[0, 1] - gbm0.predict_proba(x_test[idx])[0, 1])

ModuleNotFoundError: No module named 'lime'

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp.save_to_file('./tmp/oi.html')